In [1]:
import torch
import torch.nn as nn 
from torch.autograd import Variable
import pennylane as qml
from pennylane import numpy as np
import math 
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from PeptideDataset import *

ModuleNotFoundError: No module named 'sympy'

In [11]:
#assuming r=1,
def cartesian_to_spherical(x, y, z):
    theta = math.acos(z / 1)  
    phi = math.atan2(y, x)
    if phi < 0:
        phi += 2*np.pi
    return [theta, phi]


    


def fibonacci_sphere(n):

    points = []
    phi = math.pi * (math.sqrt(5.) - 1.)  # golden angle in radians

    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        points.append((x, y, z))
        sph_points = [cartesian_to_spherical(x,y,z) for x,y,z in points]
    
    return sph_points



In [12]:
dtype = torch.cuda.DoubleTensor if torch.cuda.is_available() else torch.DoubleTensor
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class VariationalQuantumClassifierInterface:
    def __init__(
            self,
            num_of_input,
            num_of_output,
            num_of_wires,
            num_of_layers,
            var_Q_circuit,
            var_Q_bias,
            qdevice):

        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias
        self.num_of_input = num_of_input
        self.num_of_output = num_of_output
        self.num_of_wires = num_of_wires
        self.num_of_layers = num_of_layers

        self.qdevice = qdevice

        self.dev = qml.device(self.qdevice, wires = num_of_wires)


    def set_params(self, var_Q_circuit, var_Q_bias):
        self.var_Q_circuit = var_Q_circuit
        self.var_Q_bias = var_Q_bias

    def init_params(self):
        self.var_Q_circuit = Variable(torch.tensor(0.01 * np.random.randn(self.num_of_layers, self.num_of_wires, 3), device=device).type(dtype), requires_grad=True)
        return self.var_Q_circuit

    def _statepreparation(self, angles):

        """Encoding block of circuit given angles

        Args:
            a: feature vector of rad and rad_square => np.array([rad_X_0, rad_X_1, rad_square_X_0, rad_square_X_1])
        """
        # num_of_input determines the number of rotation needed.

        for i in range(self.num_of_input):
            qml.RY(angles[i,0], wires=i)
            qml.RZ(angles[i,1], wires=i)

    def _layer(self, W):
        """ Single layer of the variational classifier.

        Args:
            W (array[float]): 2-d array of variables for one layer

        """

        # Entanglement Layer

        for i in range(self.num_of_wires):
            qml.CNOT(wires=[i, (i + 1) % self.num_of_wires])

        # Rotation Layer
        for j in range(self.num_of_wires):
            qml.Rot(W[j, 0], W[j, 1], W[j, 2], wires=j)

    def circuit(self, angles):

        @qml.qnode(self.dev, interface='torch', diff_method = "parameter-shift")
        def _circuit(var_Q_circuit, angles):
            """The circuit of the variational classifier."""
            self._statepreparation(angles)
            weights = var_Q_circuit

            for W in weights:
                self._layer(W)


            k = self.num_of_input-1
            return [qml.expval(qml.PauliZ(k))]

        return _circuit(self.var_Q_circuit, angles)



    def forward(self, angles):
        result = ((self.circuit(angles).item()))
        return torch.tensor(result, requires_grad=True)

In [17]:
vqc = VariationalQuantumClassifierInterface(
            num_of_input =12,
            num_of_output =1,
            num_of_wires=12,
            num_of_layers=2,
            var_Q_circuit=None,
            var_Q_bias = None,
            qdevice = "default.qubit")
           
fib_angles = fibonacci_sphere(18)
         

class VQCTorch(nn.Module):
    def __init__(self):
        super().__init__()

        self.q_params = nn.Parameter(0.01 * torch.randn(2, 12, 3))
    def get_angles(self, in_x):
        in_x_int = [int(item) for item in in_x.tolist()]
        angles = []
        for item in in_x_int:
            theta = fib_angles[item][0]
            phi = fib_angles[item][1]
            angles.append([theta, phi])

        return torch.tensor(angles, requires_grad=True)


    def forward(self, batch_item):
        vqc.var_Q_circuit = self.q_params
        output_batch = []

        for single_item in batch_item:
            angles = self.get_angles(single_item)

            q_out_elem = vqc.forward(angles)
            
            output_batch.append(q_out_elem)

        outputs = torch.stack(output_batch).view(len(batch_item), 1) 
        return outputs


In [18]:
params = {'batch_size' : 4, 'lr': 0.01, 'epochs': 100}

In [19]:
 
dataset = PeptideDataset()
train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print(f"Dataset: {len(dataset)}")
print(f"(Train,Test): {train_size,test_size}")
train_loader = DataLoader(dataset = train_dataset, batch_size = params['batch_size'], shuffle = True, num_workers = 2)
train_iter = iter(train_loader)
train_data = next(train_iter)
x_train, y_train = train_data

test_loader = DataLoader(dataset = train_dataset, batch_size = params['batch_size'], shuffle = True, num_workers = 2)
test_iter = iter(test_loader)
test_data = next(test_iter)
x_test, y_test = test_data


                          

Dataset: 441978
(Train,Test): (353582, 88396)


In [20]:
model = VQCTorch()

lr = params['lr']

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)  

batch_size = params['batch_size']
num_epochs = params['epochs']
total_samples = len(train_dataset)
n_iterations = math.ceil(total_samples/batch_size)
for epoch in range(num_epochs):
    print(f"EPOCH: {epoch}")
    for i, (data, target) in enumerate(train_loader):
        if i == 0:
            print(f'Inputs {data.shape} | Labels {target.shape}')
        y_predicted = model(data)
        loss = criterion(y_predicted, target)
    
        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

        #
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}|loss = {loss.item():.4f}')


        
        



EPOCH: 0
Inputs torch.Size([4, 12]) | Labels torch.Size([4, 1])


AttributeError: 'list' object has no attribute 'item'